In [1]:
import pydicom
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import numpy as np
import os
import csv
from datetime import datetime
from collections import defaultdict

DATA_FILE = r"C:\Users\maryam\Desktop\rad.csv"

THEMES = {
    "Light": {"bg": "#ffffff", "fg": "#333", "button": "#3498db", "card": "#f4f4f4", "border": "#ddd"},
    "Dark": {"bg": "#1e1e1e", "fg": "#ffffff", "button": "#00b7c2", "card": "#2c2c2c", "border": "#444"},
}

current_theme = "Light"

# CSV file init
def init_csv_file():
    if not os.path.exists(DATA_FILE):
        with open(DATA_FILE, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['PatientName', 'StudyDate', 'CTDIvol'])

# Save to CSV
patient_records = set()
def save_to_csv(patient_name, study_date, ctdi):
    key = (patient_name, study_date, ctdi)
    if key not in patient_records:
        with open(DATA_FILE, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([patient_name, study_date, ctdi])
            patient_records.add(key)

# Load CSV
def load_csv_data():
    data = []
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                data.append({
                    'PatientName': row['PatientName'],
                    'StudyDate': row['StudyDate'],
                    'CTDIvol': float(row['CTDIvol'])
                })
    return data

def parse_date(dicom_date):
    return datetime.strptime(dicom_date, "%Y%m%d")

def load_dicom_info(filepath):
    ds = pydicom.dcmread(filepath)
    patient_name = str(ds.PatientName)
    study_date = str(ds.StudyDate)
    ctdi = None
    try:
        if (0x0018, 0x9345) in ds:
            ctdi = float(ds[(0x0018, 0x9345)].value)
    except Exception as e:
        print(f"Error reading CTDIvol: {e}")
    return ds, patient_name, study_date, ctdi

def apply_theme():
    theme = THEMES[current_theme]
    root.configure(bg=theme['bg'])
    top_frame.configure(bg=theme['bg'])
    control_frame.configure(bg=theme['bg'])
    result_label.configure(bg=theme['bg'], fg=theme['fg'])
    footer.configure(bg=theme['bg'], fg=theme['fg'])

def toggle_theme():
    global current_theme
    current_theme = "Dark" if current_theme == "Light" else "Light"
    apply_theme()

def clear_all():
    for widget in image_canvas_frame.winfo_children():
        widget.destroy()
    result_label.config(text="")

def select_files():
    filepaths = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not filepaths:
        return

    clear_all()
    patient_data = defaultdict(list)
    name_set = set()

    for filepath in filepaths:
        ds, name, date, ctdi = load_dicom_info(filepath)
        if ctdi is None:
            continue

        key = (name, date, ctdi)
        if key in patient_records:
            continue

        name_set.add(name)
        save_to_csv(name, date, ctdi)
        patient_data[name].append((parse_date(date), ctdi, filepath, date))

    if len(name_set) > 1:
        messagebox.showwarning("تحذير", "الصور ليست لنفس المريض! الرجاء التأكد من اختيار صور لمريض واحد فقط.")

    image_index = 0
    for name, records in patient_data.items():
        records.sort()
        for record in records:
            date_obj, ctdi, filepath, date_str = record
            ds = pydicom.dcmread(filepath)
            img_array = ds.pixel_array
            img_pil = Image.fromarray(img_array)
            img_tk = ImageTk.PhotoImage(img_pil)

            frame = tk.Frame(image_canvas_frame, bg=THEMES[current_theme]['card'], bd=0,
                             highlightbackground=THEMES[current_theme]['border'], highlightthickness=1)
            frame.pack(fill='x', padx=20, pady=10)

            image_label = tk.Label(frame, image=img_tk, bg=THEMES[current_theme]['card'])
            image_label.image = img_tk
            image_label.pack(side="left", padx=10)

            info_text = f"Name: {name}\nDate: {date_str[:4]}-{date_str[4:6]}-{date_str[6:]}\nCTDIvol: {ctdi:.2f}"
            info_label = tk.Label(frame, text=info_text, font=("Segoe UI", 11), justify="left",
                                  bg=THEMES[current_theme]['card'], fg="green")
            info_label.pack(side="left", padx=10)

            image_index += 1

    result_label.config(text="")
    all_data = load_csv_data()
    grouped_by_patient = defaultdict(list)

    for entry in all_data:
        grouped_by_patient[entry['PatientName']].append((parse_date(entry['StudyDate']), entry['CTDIvol']))

    summary_lines = []
    for name, entries in grouped_by_patient.items():
        entries.sort()
        if not entries:
            continue
        first_date = entries[0][0]
        one_year_later = first_date.replace(year=first_date.year + 1)

        yearly_total = sum(ctdi for date, ctdi in entries if first_date <= date <= one_year_later)
        overall_total = sum(ctdi for _, ctdi in entries)

        date_range_str = f"{first_date.strftime('%Y-%m-%d')} → {one_year_later.strftime('%Y-%m-%d')}"
        summary_lines.append(f"{name} → {yearly_total:.2f} mGy from {date_range_str}")
        summary_lines.append(f"{name} → مجموع الجرعة الكلي: {overall_total:.2f} mGy")

    # عرض النتائج بتنسيق خاص
    result_label.config(text="\n\n".join(summary_lines), font=("Segoe UI", 12, "bold"), fg="green")

# GUI setup
root = tk.Tk()
root.title("DICOM CTDIvol Analyzer")
root.geometry("1200x800")
root.configure(bg=THEMES[current_theme]['bg'])

init_csv_file()

# Frames
top_frame = tk.Frame(root)
top_frame.pack(pady=10)

control_frame = tk.Frame(root)
control_frame.pack(pady=5)

canvas_frame = tk.Frame(root)
canvas_frame.pack(fill=tk.BOTH, expand=True, padx=20)

scrollbar = tk.Scrollbar(canvas_frame, orient=tk.VERTICAL)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

image_canvas = tk.Canvas(canvas_frame, yscrollcommand=scrollbar.set, bd=0, highlightthickness=0)
image_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
scrollbar.config(command=image_canvas.yview)

image_canvas_frame = tk.Frame(image_canvas)
image_canvas.create_window((0, 0), window=image_canvas_frame, anchor='nw')

def on_frame_configure(event):
    image_canvas.configure(scrollregion=image_canvas.bbox("all"))

image_canvas_frame.bind("<Configure>", on_frame_configure)

# Controls
title = tk.Label(top_frame, text="CTDIvol Viewer & Yearly Dose Calculator", font=("Segoe UI", 18, "bold"),
                 fg=THEMES[current_theme]['fg'], bg=THEMES[current_theme]['bg'])
title.pack()

style_btn = {
    'font': ("Segoe UI", 10, "bold"),
    'bd': 0,
    'fg': "white",
    'activeforeground': "white",
    'relief': tk.FLAT,
    'width': 18,
    'pady': 10,
    'cursor': "hand2"
}

select_btn = tk.Button(control_frame, text="Select DICOM Files", command=select_files,
                       bg=THEMES[current_theme]['button'], **style_btn)
select_btn.grid(row=0, column=0, padx=8)

reset_btn = tk.Button(control_frame, text="Reset for New Patient", command=clear_all,
                      bg="#e67e22", **style_btn)
reset_btn.grid(row=0, column=1, padx=8)

theme_btn = tk.Button(control_frame, text="Toggle Theme", command=toggle_theme,
                      bg="#8e44ad", **style_btn)
theme_btn.grid(row=0, column=2, padx=8)

result_label = tk.Label(root, text="", font=("Segoe UI", 12), fg="#2980b9", bg=THEMES[current_theme]['bg'])
result_label.pack(pady=10)

footer = tk.Label(root, text="Developed with ❤️ using pydicom", font=("Segoe UI", 9), fg="#aaa",
                  bg=THEMES[current_theme]['bg'])
footer.pack(side="bottom", pady=10)

apply_theme()
root.mainloop()
